In [1]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

from typing import List, Tuple
import pickle

2025-06-01 18:35:26.807801: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-01 18:35:26.983350: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-01 18:35:34.381822: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/iscb/wolfson/doririmon/anaconda3/

In [2]:
import sys
sys.path.append('../../../../')

In [3]:
with open('/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/data_for_training/03_04_with_pesto_and_coord/folds_training_dicts.pkl', 'rb') as f:
    folds_training_dicts = pickle.load(f)

2025-06-01 18:35:52.366571: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
fold = folds_training_dicts[0]
fold.keys()

dict_keys(['sizes_train', 'components_train', 'num_patches_train', 'uniprots_train', 'labels_train', 'sizes_validation', 'components_validation', 'num_patches_validation', 'uniprots_validation', 'labels_validation', 'sizes_test', 'components_test', 'num_patches_test', 'uniprots_test', 'labels_test', 'coordinates_train', 'coordinates_validation', 'coordinates_test'])

In [5]:
features = fold['components_train']
coordinates = fold['coordinates_train']
sizes = fold['sizes_train']
num_patches = fold['num_patches_train']

In [6]:
a = tf.keras.layers.Masking(mask_value=0.0)(features)
b = tf.keras.layers.Masking(mask_value=0.0)(coordinates)

In [7]:
a.shape, features.shape, b.shape

(TensorShape([76632, 10, 9]),
 TensorShape([76632, 10, 9]),
 TensorShape([76632, 10, 3]))

In [8]:
import numpy as np
np.all(a._keras_mask == b._keras_mask)

True

In [9]:
# pairwise_distances = tf.norm(
#         tf.expand_dims(b, axis=1) - tf.expand_dims(b, axis=2), axis=-1)
    

In [10]:
# pairwise_distances._keras_mask

In [11]:
# a._keras_mask[2]

In [12]:
# coordinates[0]

In [13]:
# a._keras_mask.shape

In [14]:
# features[0].shape

In [15]:
# a[0]._keras_mask

In [16]:
# coordinates.shape

In [17]:
from model import build_model

In [18]:
model = build_model([[256, 256]], 0.2, 
                    [[256, 256]], 0.2, 
                    [[256, 256]], 0.2,
                    'relu', (10, 9), 10, 256, 256, 8)

initializing projection for skip connection: 11 -> 256


In [19]:
model([features, coordinates, sizes, num_patches])

<tf.Tensor: shape=(76632, 1), dtype=float32, numpy=
array([[0.42045864],
       [0.4102031 ],
       [0.44405568],
       ...,
       [0.2794958 ],
       [0.25345257],
       [0.27265355]], dtype=float32)>

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 number_of_patches_input (Input  [(None, 11)]        0           []                               
 Layer)                                                                                           
                                                                                                  
 tf.math.argmax (TFOpLambda)    (None,)              0           ['number_of_patches_input[0][0]']
                                                                                                  
 tf.__operators__.getitem (Slic  (None, 1)           0           ['tf.math.argmax[1][0]']         
 ingOpLambda)                                                                                     
                                                                                              

In [11]:
model.compute_mask([features, coordinates, sizes, num_patches])

TypeError: Functional.compute_mask() missing 1 required positional argument: 'mask'